In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from lightgbm.basic import Dataset
import lightgbm as lgb
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from collections import OrderedDict
import sys,os

In [3]:
import sys,os
sys.path.append("./src/")
from helper import *

In [8]:
from imp import reload
import helper
reload(helper)
from helper import *

<module 'helper' from './src\\helper.py'>

In [16]:
data1027=loader("1027")
data1119=loader("1119")

In [17]:
params={
    "learning_rate":0.005,
    "n_estimators":1000
}
lgb=LGBMRegressor().set_params(**params)
models={"lgb_ratio_1027":lgb}
data1027['target']=data1027['time']/data1027['computing_mean']
train=data1027.loc[data1027.time.notnull(),:].loc[:399,:].reset_index(drop=True)
train_X=train.drop(['time','target','n_jobs','timeN1','timeN2','timeN4','timeN8','timeN16','max_iter'],axis=1)
train_y=train.target
valid=data1027.loc[400:,:].reset_index(drop=True)
valid_X=valid.drop(['time','target','n_jobs','timeN1','timeN2','timeN4','timeN8','timeN16','max_iter'],axis=1)
valid_y=valid.target
stack_train_1027ratio, stack_valid_1027ratio = stack(5,models,train_X,train_y,valid_X)
stack_train_1027ratio = stack_train_1027ratio.apply(lambda col:col * train['computing_mean'],axis=0)
stack_valid_1027ratio = stack_valid_1027ratio.apply(lambda col:col * valid['computing_mean'],axis=0)

In [18]:
params={
    "learning_rate":0.005,
    "n_estimators":1000
}
lgb=LGBMRegressor().set_params(**params)
lr=linear_model.LinearRegression()
models={"lgb_diff_1119":lgb, "lr_diff_1119":lr}
data1119['target']=data1119['time']-data1119['computing_mean']
train=data1119.loc[data1119.time.notnull(),:].loc[:399,:].reset_index(drop=True)
train_X=train.drop(['time','target','n_jobs','timeN1','timeN2','timeN4','timeN8','timeN16','max_iter'],axis=1)
train_y=train.target
valid=data1119.loc[400:,:].reset_index(drop=True)
valid_X=valid.drop(['time','target','n_jobs','timeN1','timeN2','timeN4','timeN8','timeN16','max_iter'],axis=1)
valid_y=valid.target
stack_train_1119diff, stack_valid_1119diff = stack(5,models,train_X,train_y,valid_X)
stack_train_1119diff = stack_train_1119diff.apply(lambda col:col + train['computing_mean'],axis=0)
stack_valid_1119diff = stack_valid_1119diff.apply(lambda col:col + valid['computing_mean'],axis=0)

In [19]:
params={
    "learning_rate":0.005,
    "n_estimators":1000
}
lgb=LGBMRegressor().set_params(**params)
models={"lgb_ratio_1119":lgb}

data1119['target']=data1119['time']/data1119['computing_mean']
train=data1119.loc[data1119.time.notnull(),:].loc[:399,:].reset_index(drop=True)
train_X=train.drop(['time','target','n_jobs','timeN1','timeN2','timeN4','timeN8','timeN16','max_iter'],axis=1)
train_y=train.target
valid=data1119.loc[400:,:].reset_index(drop=True)
valid_X=valid.drop(['time','target','n_jobs','timeN1','timeN2','timeN4','timeN8','timeN16','max_iter'],axis=1)
valid_y=valid.target
stack_train_1119ratio, stack_valid_1119ratio = stack(5,models,train_X,train_y,valid_X)
stack_train_1119ratio = stack_train_1119ratio.apply(lambda col:col * train['computing_mean'],axis=0)
stack_valid_1119ratio = stack_valid_1119ratio.apply(lambda col:col * valid['computing_mean'],axis=0)

In [20]:
stack_train = pd.concat([  stack_train_1027ratio, stack_train_1119ratio],axis=1)
stack_valid = pd.concat([  stack_valid_1027ratio, stack_valid_1119ratio],axis=1)
lr_stack=linear_model.LinearRegression().fit(stack_train,train.time)
test=data1027.loc[400:,:]
test.target=list(lr_stack.predict(stack_valid))
subm=pd.DataFrame({"Time":data1027.time.fillna(test.target).loc[400:]}).reset_index(drop=True).reset_index()
subm.columns=["Id","Time"]

C:\Users\zjw\Anaconda3\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [22]:
subm

,Id,Time
0,0,1.344974
1,1,10.084117
2,2,0.358246
3,3,1.609889
4,4,2.537856
5,5,8.066638
6,6,2.391719
7,7,0.628619
8,8,14.221797
9,9,0.474988


In [23]:
subm.to_csv("./submission/1202_debug.csv",index=False)
# Public : 0.23812
# Private : 0.08203